In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("nominations.db")

In [4]:
schema = conn.execute("pragma table_info(nominations);").fetchall()

In [5]:
first_ten = conn.execute("SELECT * FROM nominations LIMIT 10";).fetchall()

In [6]:
for label in schema:
    print(label)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [7]:
for row in first_ten:
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [8]:
create_query = '''
CREATE TABLE ceremonies (
id INTEGER PRIMARY KEY,
Year INTEGER,
Host TEXT
);
'''

conn.execute(create_query)

In [9]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [10]:
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"

In [11]:
conn.executemany(insert_query,years_hosts)

In [13]:
conn.execute("SELECT * FROM ceremonies LIMIT 10;").fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [14]:
conn.execute("pragma table_info(ceremonies);").fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'Year', 'INTEGER', 0, None, 0),
 (2, 'Host', 'TEXT', 0, None, 0)]

In [15]:
conn.execute("pragma foreign_keys = on;")

In [25]:
#conn.execute("DROP TABLE nominations_two;")

create_query2 = '''
CREATE TABLE nominations_two (
id INTEGER PRIMARY KEY,
category TEXT,
nominee TEXT,
movie TEXT,
character TEXT,
won INTEGER,
ceremony_id INTEGER,
FOREIGN KEY (ceremony_id) REFERENCES ceremonies(id)
);
'''

conn.execute(create_query2)

In [26]:
join_query = '''
SELECT ceremonies.id as ceremony_id, nominations.category as nominations, 
nominations.nominee as nominee, nominations.movie as movie,
nominations.character as character, nominations.won as won
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year;
'''

joined_nominations = conn.execute(join_query).fetchall()

In [27]:
joined_nominations[0]

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0)

In [30]:
insert_nominations_two = '''
INSERT INTO nominations_two(ceremony_id,category,nominee,movie,character,won)
VALUES (?,?,?,?,?,?);
'''

In [31]:
conn.executemany(insert_nominations_two,joined_nominations)

In [32]:
conn.execute("SELECT * FROM nominations_two LIMIT 5;").fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech",
  'King George VI',
  1,
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours',
  'Aron Ralston',
  0,
  1)]

In [33]:
conn.execute("DROP TABLE nominations")

In [34]:
conn.execute("ALTER TABLE nominations_two RENAME TO nominations")

In [79]:
create_movies = '''
CREATE TABLE movies (
id integer primary key,
movie text
);
'''
create_actors = '''
CREATE TABLE actors (
id integer primary key,
actor text
);
'''

create_movies_actors = '''
CREATE TABLE movies_actors (
id integer primary key,
movie_id integer references movies(id),
actor_id integer references actors(id)
);
'''

for query in [create_movies,create_actors,create_movies_actors]:
    conn.execute(query)